## PGA Draftkings Notebook
Use [PGA Website](https://www.pgatour.com/tournaments/schedule.html) to look up tournament info and fill out first USER INPUT block below. (Fetch/XHR from Network tab, Paylod sub-tab)

Looks like the tournament ID is also in the address bar.

### User Input

In [2]:
# === USER INPUTS ===
# Old Tournament
old_tournament_name = "Truist Championship"
tournament_date = "5/11/2025"
old_course = "The Philadelphia Cricket Club"
tournament_id = "R2025480"

# New Tournament
new_tournament_name = "PGA Championship"
new_course = "Quail Hollow Club"


# === LIBRARIES AND VARIABLES ===
# Import necessary libraries
import requests
import pandas as pd
from datetime import datetime
import sqlite3 as sql
import numpy as np
from numpy import nan
import os
import importlib
import utils.db_utils
from utils.db_utils import TOURNAMENT_NAME_MAP, PLAYER_NAME_MAP

# ===============================

tournament_config = {
    "old": {
        "name": old_tournament_name,
        "date": tournament_date,
        "course": old_course,
        "id": tournament_id
    },
    "new": {
        "name": new_tournament_name,
        "course": new_course,
        "quoted_course": f'"{new_course}"',
        "quoted_name": f"'{new_tournament_name}'"
    }
}

### Update Database

#### Old Tournament

In [3]:
importlib.reload(utils.db_utils)  # Only needed if you're actively editing db_utils.py
from utils.db_utils import update_tournament_results

# Change these each year!!
season = 2025
year = 20250  # Unique GraphQL year distinguishing number in case of multiple per year

# Run the update
db_path = "data/golf.db"  # Or use os.path.join("data", "golf.db")
tournDf = update_tournament_results(tournament_config, db_path, season, year)

# Show just the most recent tournament added for confirmation
conn = sql.connect(db_path)
query = f"""
SELECT *
FROM tournaments
WHERE TOURN_ID = '{tournament_config['old']['id']}'
  AND ENDING_DATE = '{datetime.strptime(tournament_config['old']['date'], '%m/%d/%Y').date()}'
"""

recent = pd.read_sql_query(query, conn)
conn.close()

recent.head()

📦 Preparing payload for tournament ID: R2025480, year: 20250
📬 Sending request to PGA Tour API...
✅ API request succeeded.
🔍 Found 72 players in response.
💾 Connecting to database...
✅ Tournament results for 'Truist Championship' added to data/golf.db


,SEASON,ENDING_DATE,TOURN_ID,TOURNAMENT,COURSE,PLAYER,POS,ROUNDS:1,ROUNDS:2,ROUNDS:3,ROUNDS:4,OFFICIAL_MONEY,FEDEX_CUP_POINTS,FINAL_POS
0,2025,2025-05-11,R2025480,Truist Championship,The Philadelphia Cricket Club,Sepp Straka,1,-7,-3,-4,-2,"$3,600,000.00",700.000,1
1,2025,2025-05-11,R2025480,Truist Championship,The Philadelphia Cricket Club,Justin Thomas,T2,-4,-3,-4,-3,"$1,760,000.00",375.000,2
2,2025,2025-05-11,R2025480,Truist Championship,The Philadelphia Cricket Club,Shane Lowry,T2,-6,-5,-3,E,"$1,760,000.00",375.000,2
3,2025,2025-05-11,R2025480,Truist Championship,The Philadelphia Cricket Club,Patrick Cantlay,T4,-5,-2,E,-5,"$826,666.67",300.000,4
4,2025,2025-05-11,R2025480,Truist Championship,The Philadelphia Cricket Club,Jacob Bridgeman,T4,+1,-3,-5,-5,"$826,666.67",300.000,4


#### Stats

In [3]:
importlib.reload(utils.db_utils)
from utils.db_utils import update_season_stats  # <- This line is essential

# Change these each year!!
statsYear = 2025

stats_df = update_season_stats(statsYear, db_path)
stats_df.head()

✅ Stats for season 2025 updated in database.


,SEASON,PLAYER,SGTTG_RANK,SG:TTG,SGOTT_RANK,SG:OTT,SGAPR_RANK,SG:APR,SGATG_RANK,SG:ATG,...,DRIVING_ACCURACY,GIR_RANK,GIR,SCRAMBLING_RANK,SCRAMBLING,OWGR_RANK,OWGR,PAR_3_RANK,PAR_4_RANK,PAR_5_RANK
0,2025,Brendan Jones,NaN,None,NaN,None,NaN,None,NaN,None,...,None,NaN,None,NaN,None,996.0,.0792,NaN,NaN,NaN
1,2025,Victor Pastor Rufian,NaN,None,NaN,None,NaN,None,NaN,None,...,None,NaN,None,NaN,None,998.0,.0790,NaN,NaN,NaN
2,2025,Daniel Miernicki,NaN,None,NaN,None,NaN,None,NaN,None,...,None,NaN,None,NaN,None,985.0,.0795,NaN,NaN,NaN
3,2025,Gavin Cohen,172.0,-2.064,149.0,-0.423,183.0,-1.811,69.0,0.171,...,53.57%,178.0,63.89%,104.0,61.54%,NaN,None,NaN,NaN,NaN
4,2025,Gudmundur Kristjansson,NaN,None,NaN,None,NaN,None,NaN,None,...,None,NaN,None,NaN,None,995.0,.0777,NaN,NaN,NaN


#### Odds

Manual Fix! Odds name cleanup (only needed when joins fail)

Make sure to update the dictionaries in db_utils.py if new names need to be added.

In [ ]:
from utils.db_utils import clean_odds_names, PLAYER_NAME_MAP, TOURNAMENT_NAME_MAP

clean_odds_names(db_path, TOURNAMENT_NAME_MAP, PLAYER_NAME_MAP)

## Update Database Structure (Temporary)

In [4]:
import pandas as pd
import sqlite3
from sqlalchemy import create_engine
# importlib.reload(utils.schema)
from utils.schema import stats_table, metadata

db_path = "data/golf.db"
engine = create_engine(f"sqlite:///{db_path}")

# Step 1: Load existing stats table
with sqlite3.connect(db_path) as conn:
    old_df = pd.read_sql("SELECT * FROM stats", conn)

# Rename old colon-based and malformed rank columns to match new schema
stat_column_renames = {
    "SG:TTG": "SGTTG",
    "SG:OTT": "SGOTT",
    "SG:APR": "SGAPR",
    "SG:ATG": "SGATG",
    "SG:P": "SGP",
    "PAR 3": "PAR_3",
    "PAR 4": "PAR_4",
    "PAR 5": "PAR_5",
    "PAR3_RANK": "PAR_3_RANK",
    "PAR4_RANK": "PAR_4_RANK",
    "PAR5_RANK": "PAR_5_RANK"
}

old_df = old_df.rename(columns=stat_column_renames)

# Step 2: Add missing columns if needed
required_columns = [col.name for col in stats_table.columns]
for col in required_columns:
    if col not in old_df.columns:
        old_df[col] = None

# Step 3: Deduplicate and clean
deduped_df = old_df.drop_duplicates(subset=["SEASON", "PLAYER"]).copy()

deduped_df.loc[:, "SEASON"] = deduped_df["SEASON"].astype(int)

# Step 4: Overwrite with new schema
with engine.begin() as conn:
    metadata.drop_all(conn, tables=[stats_table])
    metadata.create_all(conn)
    deduped_df.to_sql("stats", conn, index=False, if_exists="append")

print("✅ Migration complete: 'stats' table updated.")


✅ Migration complete: 'stats' table updated.


In [5]:
import pandas as pd
import sqlite3
from sqlalchemy import create_engine
from utils.schema import tournaments_table, metadata

# Set up database path and SQLAlchemy engine
db_path = "data/golf.db"
engine = create_engine(f"sqlite:///{db_path}")

# Step 1: Load original data from the old tournaments table
with sqlite3.connect(db_path) as conn:
    old_df = pd.read_sql("SELECT * FROM tournaments", conn)

# Step 2: Ensure required columns exist
required_columns = [col.name for col in tournaments_table.columns]
for col in required_columns:
    if col not in old_df.columns:
        old_df[col] = None

# Step 3: Deduplicate using the new composite primary key
deduped_df = old_df.drop_duplicates(subset=["SEASON", "TOURNAMENT", "PLAYER"]).copy()

# Step 4: Prepare the new table contents
try:
    deduped_df.loc[:, "SEASON"] = deduped_df["SEASON"].astype(int)
    deduped_df.loc[:, "FINAL_POS"] = pd.to_numeric(deduped_df["FINAL_POS"], errors="coerce")
    deduped_df.loc[:, "ENDING_DATE"] = pd.to_datetime(deduped_df["ENDING_DATE"]).dt.date
except Exception as e:
    print("❌ Data transformation failed:", e)
    engine.dispose()
    raise

# Step 5: Only drop/create if transformation succeeds
with engine.begin() as conn:
    metadata.drop_all(conn, tables=[tournaments_table])
    metadata.create_all(conn)
    deduped_df.to_sql("tournaments", conn, index=False, if_exists="append")

# Step 6: Dispose the engine to release file lock
engine.dispose()

print("✅ Migration complete: Data inserted into refactored 'tournaments' table.")


✅ Migration complete: Data inserted into refactored 'tournaments' table.
